##### NLTK and Random Forest
###### Author: Desi Nikolova
###### Date: 27th June, 2024

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import nltk
nltk.download()
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [2]:
### Load the dataset 
import pandas as pd
df = pd.read_csv('GroceryDataset.csv')
df.head()
df.columns
df.dtypes
columns = df.columns

#Convert price to float 
s = pd.Series(df['Price'], dtype="string")
df['Price'] = s
df['Price'].isna().sum()

def removeCurrency(x):
    #x = str(x)
    if pd.isna(x):
        return "0"
    x = x.lstrip('$')
    return x
df['Price'] = df['Price'].apply(removeCurrency)

def discountOnOff(x):
    #x = str(x)
    if x == "No Discount":
        return 0
    return 1

s = pd.Series(df['Price'], dtype="float")
df['Price'] = s

#Encode all variables 
#from sklearn.preprocessing import OneHotEncoder
#enc = OneHotEncoder()
#one_hot_encoded = enc.fit_transform(df[['Sub Category']])
#one_hot_df = pd.DataFrame(one_hot_encoded, 
                          #columns=['Sub Category'])

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['Sub Category'] = labelencoder.fit_transform(df['Sub Category'])
df['Sub Category'] = labelencoder.fit_transform(df['Sub Category'])
df['Discount'] =  df['Discount'].apply(discountOnOff)
print(df.shape)
total = df['Discount'].sum()
print(total, "discounts")

#"Rating"	"Title"	"Feature"	"Product Description"

training_data_size = df["Product Description"].size
testing_data_size = df["Product Description"].size
print(training_data_size)
print(testing_data_size)

print(df.head())

(1757, 8)
131 discounts
1757
1757


   Sub Category    Price  Discount  \
0             0   56.99          0   
1             0  159.99          0   
2             0   44.99          0   
3             0   39.99          0   
4             0   59.99          0   

                                            Rating  \
0   Rated 4.3 out of 5 stars based on 265 reviews.   
1       Rated 5 out of 5 stars based on 1 reviews.   
2   Rated 4.1 out of 5 stars based on 441 reviews.   
3  Rated 4.7 out of 5 stars based on 9459 reviews.   
4   Rated 4.5 out of 5 stars based on 758 reviews.   

                                               Title Currency  \
0  David’s Cookies Mile High Peanut Butter Cake, ...        $   
1  The Cake Bake Shop 8" Round Carrot Cake (16-22...        $   
2  St Michel Madeleine, Classic French Sponge Cak...        $   
3  David's Cookies Butter Pecan Meltaways 32 oz, ...        $   
4  David’s Cookies Premier Chocolate Cake, 7.2 lb...        $   

                                             Feature  \

In [4]:
def clean_text_data(data_point, data_size):
    review_soup = BeautifulSoup(data_point)
    review_text = review_soup.get_text()
    review_letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    review_lower_case = review_letters_only.lower()
    review_words = review_lower_case.split()
    stop_words = stopwords.words("english")

    meaningful_words = [x for x in review_words if x not in stop_words]
    if((i)%2000 == 0):
        print("Cleaned %d of %d data (%d %%)." % (i, data_size, ((i)/data_size)*100))
    return( " ".join( meaningful_words))

import numpy as np
for i in range(training_data_size):
    val = df["Product Description"][i]
    val_rating = df["Rating"][i]
    val_feature = df["Feature"][i]
    val_title = df["Title"][i]
    if pd.isna(val):
        val = ""
    if pd.isna(val_rating):
        val_rating = ""
    if pd.isna(val_feature):
        val_feature = ""
    if pd.isna(val_title):
        val_title = ""
    review_soup = BeautifulSoup(val)
    df["Product Description"][i] = clean_text_data(val, training_data_size)
    df["Rating"][i] = clean_text_data(val_rating, training_data_size)
    df["Feature"][i] = clean_text_data(val_feature, training_data_size)
    df["Title"][i] = clean_text_data(val_title, training_data_size)
    print(df["Product Description"][i])
    print("Cleaning training completed!")

Cleaned 0 of 1757 data (0 %).
Cleaned 0 of 1757 data (0 %).
Cleaned 0 of 1757 data (0 %).
Cleaned 0 of 1757 data (0 %).
cake dessert epicure die top selling cake fudge brownie base layered velvety smooth peanut butter mousse rich chocolate cake topped brownie chunks handful peanut butter chips drizzled fudge cake thoughtful gift idea perfect family friends coworkers anyone care life generously sized precut slices cake lover dreams come true includes measures diameterweighs lbs servings ou certified trusted kosher certification u natural added preservativessome products may contain nuts facility nut free facility result possible product may come contact nut nut oils
Cleaning training completed!
due perishable nature item orders ship weekend orders delivered wednesday thursday friday minimum delivery time business days plate included gwendolyn rogers cake bake shop famous handcrafting magnificent delicious cakes desserts award winning restaurants cake arrives beautifully packaged bakery 

/var/folders/cr/5jcyqtbj695fsfwff1yqr_bc0000gn/T/ipykernel_7324/188135678.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Product Description"][i] = clean_text_data(val, training_data_size)
/var/folders/cr/5jcyqtbj695fsfwff1yqr_bc0000gn/T/ipykernel_7324/188135678.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Rating"][i] = clean_text_data(val_rating, training_data_size)
/var/folders/cr/5jcyqtbj695fsfwff1yqr_bc0000gn/T/ipykernel_7324/188135678.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.o

kosher gluten free calories per pieces count total net weight oz
Cleaning training completed!
goldbears gummi candyassortment pineapple lemon raspberry orange strawberry oz bags count
Cleaning training completed!
duos mints strawberry mint oz count tins
Cleaning training completed!
longing bold fruit flavors skittles rainbow get enough unexplainably juicy flavors starburst better stock family favorite treats avoid finding facing candy shortage perfect holiday gifting simply sharing friends variety pack includes skittles original sour wild berry flavors starburst original favereds tropical flavors consider everything need keep good going package includes skittles starburst candy variety pack single packs skittles bite size chewy candies together starburst fruit chewsperfect parties easter christmas valentine day halloween everyday snackthis variety pack includes skittles original sour wild berry flavors starburst original favereds tropical flavorstwo kinds candies one box
Cleaning train

/var/folders/cr/5jcyqtbj695fsfwff1yqr_bc0000gn/T/ipykernel_7324/188135678.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Title"][i] = clean_text_data(val_title, training_data_size)
/var/folders/cr/5jcyqtbj695fsfwff1yqr_bc0000gn/T/ipykernel_7324/188135678.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Product Description"][i] = clean_text_data(val, training_data_size)
/var/folders/cr/5jcyqtbj695fsfwff1yqr_bc0000gn/T/ipykernel_7324/188135678.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org

cafe espresso whole coffee beans lbs arabica beans kosher preparation method espresso machine drip coffee machine french press pour moka cold brew espresso italiano selection blends perfected lavazza enjoy real taste aroma rich flavorful espresso home expertise heritage lavazza master blenders coffee experience character favorite coffee brew arabica blend central south america highly aromatic notes rich body enjoy distinctive flavor lavazza espresso
Cleaning training completed!

Cleaning training completed!
prop warning item may available local warehouse lower non delivered price named city known romance easy fall love first sip multi region blend winning hearts decades originally created dessert coffee seattle restaurant quickly became hit stores well balanced rich flavors dark cocoa caramelized sugar pairs perfectly anything chocolate nestl uses starbucks trademarks license features premium coffee k cup starbucks delivers exceptional coffee cup cup count k cup pod box caff verona dar

...

/var/folders/cr/5jcyqtbj695fsfwff1yqr_bc0000gn/T/ipykernel_7324/188135678.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Title"][i] = clean_text_data(val_title, training_data_size)
/var/folders/cr/5jcyqtbj695fsfwff1yqr_bc0000gn/T/ipykernel_7324/188135678.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Product Description"][i] = clean_text_data(val, training_data_size)
/var/folders/cr/5jcyqtbj695fsfwff1yqr_bc0000gn/T/ipykernel_7324/188135678.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org

pfas added compostable bpi certified plate made bagasse discarded sugarcane fiber sugarcane annually renewable resource bagasse sugarcane fiber remaining extraction juice sugarcane generally bagasse discarded burnt creating pollution diameter plates microwave refrigerator safe use hot cold items light oil grease resistant best alternative virgin paper plastic styro foam plates upscale sustainable dinnerware microwave oily foods natural brown surface provides brilliant exterior appearance recommended composted commercial composting facility check see appropriate facility exists area
Cleaning training completed!
pfas added compostable bpi certified plate made bagasse discarded sugarcane fiber sugarcane annually renewable resource bagasse sugarcane fiber remaining extraction juice sugarcane generally bagasse discarded burnt creating pollution diameter plates microwave refrigerator safe use hot cold items light oil grease resistant best alternative virgin paper plastic styro foam plates up

/var/folders/cr/5jcyqtbj695fsfwff1yqr_bc0000gn/T/ipykernel_7324/188135678.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Rating"][i] = clean_text_data(val_rating, training_data_size)
/var/folders/cr/5jcyqtbj695fsfwff1yqr_bc0000gn/T/ipykernel_7324/188135678.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Feature"][i] = clean_text_data(val_feature, training_data_size)
/var/folders/cr/5jcyqtbj695fsfwff1yqr_bc0000gn/T/ipykernel_7324/188135678.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/p

craisins net weight oz
Cleaning training completed!
nilla wafers count oz pack
Cleaning training completed!
juicy fruit snacks variety pack berry blast mixed fruit see juice taste fruit fruit first ingredient naturally sourced colors flavors oz pouch ct
Cleaning training completed!
oz
Cleaning training completed!
net weight oz
Cleaning training completed!
total net weight oz baked usa
Cleaning training completed!
madeleine brownie every bite right
Cleaning training completed!
counttotal net weight oz
Cleaning training completed!
pretzel twists swirl buttery sweet spicy tang bite
Cleaning training completed!
grandma vanilla sandwich cr cookies naturally artificially flavored
Cleaning training completed!
g protein g sugar flavor low sodium total net weight oz
Cleaning training completed!
original net weight oz
Cleaning training completed!
excellent source vitamins c e oz pouches counttotal net weight lbs
Cleaning training completed!
takis hot chili pepper lime tortilla chips individually

/var/folders/cr/5jcyqtbj695fsfwff1yqr_bc0000gn/T/ipykernel_7324/188135678.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Feature"][i] = clean_text_data(val_feature, training_data_size)
/var/folders/cr/5jcyqtbj695fsfwff1yqr_bc0000gn/T/ipykernel_7324/188135678.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Title"][i] = clean_text_data(val_title, training_data_size)
/var/folders/cr/5jcyqtbj695fsfwff1yqr_bc0000gn/T/ipykernel_7324/188135678.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pan

In [5]:
import nltk
import sklearn

print('The nltk version is {}.'.format(nltk.__version__))
print('The scikit-learn version is {}.'.format(sklearn.__version__))


The nltk version is 3.8.1.
The scikit-learn version is 1.4.2.


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000)

#"Product Description","Rating","Feature","Title"
X_train, X_cv, Y_train, Y_cv = train_test_split(df["Product Description"], 
                                                df["Discount"], 
                                                test_size = 0.3, 
                                                random_state=42)
X_train = vectorizer.fit_transform(X_train)
X_train = X_train.toarray()
#print(X_train.shape)

X_cv = vectorizer.transform(X_cv)
X_cv = X_cv.toarray()
#print(X_cv.shape)

forest = RandomForestClassifier()
forest = forest.fit( X_train, Y_train)
predictions = forest.predict(X_cv)
print("Accuracy: ", accuracy_score(Y_cv, predictions))

#X_test = vectorizer.transform(df_test["review"])
##X_test = X_test.toarray()
#print(X_test.shape)


Accuracy:  0.9526515151515151


In [7]:
vocab = vectorizer.get_feature_names_out()
#print(f"Printing first 100 vocabulary samples:\n{vocab[:100]}")
distribution = np.sum(X_train, axis=0)
print("Printing first 100 vocab-dist pairs:")
for tag, count in zip(vocab[:100], distribution[:100]): 
    print(count, tag)

Printing first 100 vocab-dist pairs:
4 aa
5 aaa
7 aafco
5 abf
2 ability
9 able
2 abounds
2 abruptly
2 absence
3 absolute
7 absolutely
8 absorb
3 absorbency
8 absorbent
2 absorption
7 abundance
4 abundant
3 accelerates
3 accent
2 accented
2 accents
4 accepts
6 access
3 accident
4 accompanied
5 accompaniments
2 accompany
23 according
3 accredited
2 accurate
2 acerola
4 acesulfame
4 acetum
2 aches
4 achieve
6 achieving
40 acid
9 acidity
14 acidophilus
28 acids
8 acipenser
6 acorn
7 acquired
4 acres
18 across
2 acting
4 actions
2 activated
17 active
2 activewear
5 activities
4 activity
69 add
104 added
2 addedno
2 addedraised
2 addiction
4 addictive
28 adding
35 addition
9 additional
3 additionally
7 additions
8 additives
4 additivesnon
8 adds
4 adherence
5 adheres
2 adjustable
3 adjusts
3 administered
3 adorable
4 adore
3 adorn
5 adult
5 adults
8 advance
6 advanced
4 advantage
5 adventure
2 afar
6 affordable
10 africa
11 african
3 afternoon
7 aftertaste
5 age
51 aged
3 agent
5 agents
3 ag

In [8]:
forest = RandomForestClassifier()
forest = forest.fit( X_train, Y_train)

predictions = forest.predict(X_cv)
print("Accuracy: ", accuracy_score(Y_cv, predictions))

Accuracy:  0.9507575757575758
